## Baseline 相关实验

### 数据预处理

In [1]:
# Import the libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

# Algorithms of classification
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
# SVC
from sklearn.svm import SVC
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
# Random Forest
from sklearn.ensemble import RandomForestClassifier
# Logistic Regression
from sklearn.linear_model import LogisticRegression
# K-NN
from sklearn.neighbors import KNeighborsClassifier
# MLP
from sklearn.neural_network import MLPClassifier
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

# Import of library for data split
from sklearn.model_selection import train_test_split
# Confusion matrix
from sklearn.metrics import confusion_matrix, mean_squared_error
# Classification report
from sklearn.metrics import classification_report
# Import the libraries for the metrics
from sklearn.metrics import precision_score, recall_score, f1_score

# Library to calculate the mean and standard deviation used in the characteristics
from sklearn.preprocessing import StandardScaler
# Library Grid Search
from sklearn.model_selection import GridSearchCV

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from time import time
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, roc_curve, accuracy_score, f1_score, roc_auc_score, classification_report
from astropy.table import Table
from sklearn.metrics import roc_auc_score

df = pd.read_csv('E:\zhw_git\students-performance-prediction-UCI\dataset\dataset_math\student-data-train-valid.csv')
df_test = pd.read_csv('E:\zhw_git\students-performance-prediction-UCI\dataset\dataset_math\student-data-test.csv')

In [3]:
# mapping strings to numeric values(df_test_test):
def numerical_data_df_test():
    df_test['school'] = df_test['school'].map({'GP': 0, 'MS': 1})
    df_test['sex'] = df_test['sex'].map({'M': 0, 'F': 1})
    df_test['address'] = df_test['address'].map({'U': 0, 'R': 1})
    df_test['famsize'] = df_test['famsize'].map({'LE3': 0, 'GT3': 1})
    df_test['Pstatus'] = df_test['Pstatus'].map({'T': 0, 'A': 1})
    df_test['Mjob'] = df_test['Mjob'].map({'teacher': 0, 'health': 1, 'services': 2, 'at_home': 3, 'other': 4})
    df_test['Fjob'] = df_test['Fjob'].map({'teacher': 0, 'health': 1, 'services': 2, 'at_home': 3, 'other': 4})
    df_test['reason'] = df_test['reason'].map({'home': 0, 'reputation': 1, 'course': 2, 'other': 3})
    df_test['guardian'] = df_test['guardian'].map({'mother': 0, 'father': 1, 'other': 2})
    df_test['schoolsup'] = df_test['schoolsup'].map({'no': 0, 'yes': 1})
    df_test['famsup'] = df_test['famsup'].map({'no': 0, 'yes': 1})
    df_test['paid'] = df_test['paid'].map({'no': 0, 'yes': 1})
    df_test['activities'] = df_test['activities'].map({'no': 0, 'yes': 1})
    df_test['nursery'] = df_test['nursery'].map({'no': 0, 'yes': 1})
    df_test['higher'] = df_test['higher'].map({'no': 0, 'yes': 1})
    df_test['internet'] = df_test['internet'].map({'no': 0, 'yes': 1})
    df_test['romantic'] = df_test['romantic'].map({'no': 0, 'yes' : 1})
    df_test['passed'] = df_test['passed'].map({'no': 0, 'yes': 1})
    # reorder dataframe col_testumns :
    col_test = df_test['passed']
    # type(col_test) col_test的类型为df_test
    del df_test['passed']
    df_test['passed'] = col_test
    
# feature scaling will allow the algorithm to converge faster, large data will have same scal
#特征缩放操作，使得收敛速度更快，处于同一个数据维度
def feature_scaling(df_test):
    for i in df_test:
        col_test = df_test[i]
        # let's choose col_testumns that have large values
        if(np.max(col_test)>6):
            Max = max(col_test)
            Min = min(col_test)
            mean = np.mean(col_test)
            col_test  = (col_test-mean)/(Max)
            df_test[i] = col_test
        elif(np.max(col_test)<6):
            col_test = (col_test-np.min(col_test))
            col_test /= np.max(col_test)
            df_test[i] = col_test

In [4]:
numerical_data_df_test()
df_test

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,0,0,17,0,1,0,2,1,4,4,...,1,0,4,5,1,1,1,3,2,1
1,1,0,18,1,0,0,1,2,3,2,...,1,1,4,3,3,2,3,3,3,1
2,0,0,18,1,0,0,3,3,4,2,...,1,1,4,3,3,1,3,5,8,0
3,0,1,16,0,1,1,2,1,4,4,...,1,1,5,3,4,1,1,2,8,1
4,1,0,20,0,0,1,2,2,2,2,...,0,0,5,5,4,4,5,4,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,1,1,17,1,1,0,1,2,4,2,...,0,0,3,2,2,1,2,3,0,1
75,0,1,15,0,0,0,3,2,2,4,...,1,0,4,4,4,1,1,5,10,0
76,0,0,15,1,1,0,2,1,1,2,...,1,1,5,4,2,1,1,5,8,0
77,0,1,15,0,1,0,4,2,1,2,...,1,1,3,2,2,1,1,5,2,1


In [5]:
feature_scaling(df_test)
df_test

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,0.0,0.0,0.024684,0.0,1.0,0.0,0.50,0.25,1.00,1.0,...,1.0,0.0,0.75,1.00,0.00,0.00,0.00,0.50,-0.076949,1.0
1,1.0,0.0,0.074684,1.0,0.0,0.0,0.25,0.50,0.75,0.5,...,1.0,1.0,0.75,0.50,0.50,0.25,0.50,0.50,-0.050633,1.0
2,0.0,0.0,0.074684,1.0,0.0,0.0,0.75,0.75,1.00,0.5,...,1.0,1.0,0.75,0.50,0.50,0.00,0.50,1.00,0.080946,0.0
3,0.0,1.0,-0.025316,0.0,1.0,1.0,0.50,0.25,1.00,1.0,...,1.0,1.0,1.00,0.50,0.75,0.00,0.00,0.25,0.080946,1.0
4,1.0,0.0,0.174684,0.0,0.0,1.0,0.50,0.50,0.50,0.5,...,0.0,0.0,1.00,1.00,0.75,0.75,1.00,0.75,0.159893,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,1.0,1.0,0.024684,1.0,1.0,0.0,0.25,0.50,1.00,0.5,...,0.0,0.0,0.50,0.25,0.25,0.00,0.25,0.50,-0.129580,1.0
75,0.0,1.0,-0.075316,0.0,0.0,0.0,0.75,0.50,0.50,1.0,...,1.0,0.0,0.75,0.75,0.75,0.00,0.00,1.00,0.133578,0.0
76,0.0,0.0,-0.075316,1.0,1.0,0.0,0.50,0.25,0.25,0.5,...,1.0,1.0,1.00,0.75,0.25,0.00,0.00,1.00,0.080946,0.0
77,0.0,1.0,-0.075316,0.0,1.0,0.0,1.00,0.50,0.25,0.5,...,1.0,1.0,0.50,0.25,0.25,0.00,0.00,1.00,-0.076949,1.0


In [6]:
# mapping strings to numeric values(df):
def numerical_data_df():
    df['school'] = df['school'].map({'GP': 0, 'MS': 1})
    df['sex'] = df['sex'].map({'M': 0, 'F': 1})
    df['address'] = df['address'].map({'U': 0, 'R': 1})
    df['famsize'] = df['famsize'].map({'LE3': 0, 'GT3': 1})
    df['Pstatus'] = df['Pstatus'].map({'T': 0, 'A': 1})
    df['Mjob'] = df['Mjob'].map({'teacher': 0, 'health': 1, 'services': 2, 'at_home': 3, 'other': 4})
    df['Fjob'] = df['Fjob'].map({'teacher': 0, 'health': 1, 'services': 2, 'at_home': 3, 'other': 4})
    df['reason'] = df['reason'].map({'home': 0, 'reputation': 1, 'course': 2, 'other': 3})
    df['guardian'] = df['guardian'].map({'mother': 0, 'father': 1, 'other': 2})
    df['schoolsup'] = df['schoolsup'].map({'no': 0, 'yes': 1})
    df['famsup'] = df['famsup'].map({'no': 0, 'yes': 1})
    df['paid'] = df['paid'].map({'no': 0, 'yes': 1})
    df['activities'] = df['activities'].map({'no': 0, 'yes': 1})
    df['nursery'] = df['nursery'].map({'no': 0, 'yes': 1})
    df['higher'] = df['higher'].map({'no': 0, 'yes': 1})
    df['internet'] = df['internet'].map({'no': 0, 'yes': 1})
    df['romantic'] = df['romantic'].map({'no': 0, 'yes' : 1})
    df['passed'] = df['passed'].map({'no': 0, 'yes': 1})
    # reorder dataframe columns :
    col = df['passed']
    # type(col) col的类型为df
    del df['passed']
    df['passed'] = col

# feature scaling will allow the algorithm to converge faster, large data will have same scal
#特征缩放操作，使得收敛速度更快，处于同一个数据维度
def feature_scaling(df):
    for i in df:
        col = df[i]
        # let's choose columns that have large values
        if(np.max(col)>6):
            Max = max(col)
            Min = min(col)
            mean = np.mean(col)
            col  = (col-mean)/(Max)
            df[i] = col
        elif(np.max(col)<6):
            col = (col-np.min(col))
            col /= np.max(col)
            df[i] = col

In [7]:
numerical_data_df()
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,0,1,17,0,1,0,1,1,3,4,...,1,0,5,3,3,1,1,3,4,0
1,0,1,15,0,0,0,1,1,3,4,...,1,0,4,3,2,2,3,3,10,1
2,0,1,16,0,1,0,3,3,4,4,...,0,0,4,3,2,1,2,5,4,1
3,0,0,16,0,0,0,2,2,4,4,...,1,0,4,4,4,1,1,3,0,1
4,0,1,17,0,1,1,4,4,4,0,...,0,0,4,1,4,1,1,1,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,1,1,19,1,1,0,2,3,2,4,...,1,0,5,4,2,1,2,5,0,0
312,1,1,18,0,0,0,3,1,0,2,...,1,0,4,3,4,1,1,1,0,0
313,1,1,18,0,1,0,1,1,4,4,...,0,0,1,1,1,1,1,5,0,0
314,1,0,21,1,1,0,1,1,4,4,...,0,0,5,5,3,3,3,3,3,0


In [8]:
feature_scaling(df)
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,0.0,1.0,0.011651,0.0,1.0,0.0,0.25,0.25,0.75,1.00,...,1.0,0.0,1.00,0.50,0.50,0.00,0.00,0.5,-0.025401,0.0
1,0.0,1.0,-0.079258,0.0,0.0,0.0,0.25,0.25,0.75,1.00,...,1.0,0.0,0.75,0.50,0.25,0.25,0.50,0.5,0.054599,1.0
2,0.0,1.0,-0.033803,0.0,1.0,0.0,0.75,0.75,1.00,1.00,...,0.0,0.0,0.75,0.50,0.25,0.00,0.25,1.0,-0.025401,1.0
3,0.0,0.0,-0.033803,0.0,0.0,0.0,0.50,0.50,1.00,1.00,...,1.0,0.0,0.75,0.75,0.75,0.00,0.00,0.5,-0.078734,1.0
4,0.0,1.0,0.011651,0.0,1.0,1.0,1.00,1.00,1.00,0.00,...,0.0,0.0,0.75,0.00,0.75,0.00,0.00,0.0,0.001266,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,1.0,1.0,0.102560,1.0,1.0,0.0,0.50,0.75,0.50,1.00,...,1.0,0.0,1.00,0.75,0.25,0.00,0.25,1.0,-0.078734,0.0
312,1.0,1.0,0.057106,0.0,0.0,0.0,0.75,0.25,0.00,0.50,...,1.0,0.0,0.75,0.50,0.75,0.00,0.00,0.0,-0.078734,0.0
313,1.0,1.0,0.057106,0.0,1.0,0.0,0.25,0.25,1.00,1.00,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,1.0,-0.078734,0.0
314,1.0,0.0,0.193470,1.0,1.0,0.0,0.25,0.25,1.00,1.00,...,0.0,0.0,1.00,1.00,0.50,0.50,0.50,0.5,-0.038734,0.0


In [13]:
data = df.to_numpy()
data_test = df_test.to_numpy()
# n = data.shape[1]
x = data[:, : -1]
y = data[:,-1]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=4)
x_test_true = data_test[:, :-1]
y_test_true = data_test[:, -1]

### Logistic Regression

In [63]:
logisticRegr = LogisticRegression(C=1)

In [ ]:
logisticRegr.fit(x_train,y_train)

#### 评估指标

In [ ]:
y_pred_true=logisticRegr.predict(x_test_true)
y_pred_true

In [ ]:
# 打印数据报告
print(classification_report(y_test_true, y_pred_true,digits=3))

print()

In [ ]:
mse = mean_squared_error(y_test_true, y_pred_true)
rmse = np.sqrt(mse)
print("RMSE为：", rmse)

In [ ]:
# 混淆矩阵
cm = confusion_matrix(y_test_true, y_pred_true)
sns.heatmap(cm,annot=True)

In [ ]:
# ROC曲线
fpositif, tpositif, thresholds = roc_curve(y_test_true, y_pred_true)
plt.plot([0,1],[0,1],'--')
plt.plot(fpositif,tpositif, label='LogisticRegr')
plt.xlabel('false positif')
plt.ylabel('true positif')
plt.title('LogisticRegr ROC curve')
p=plt.show()

### k-nearest neighbors

In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
x_test_true = scaler.transform(x_test_true)

knn = KNeighborsClassifier()

# Parameters
n_neighbors = [1, 3, 5, 10]
weights = ['uniform','distance']
algorithm = ['auto','ball_tree','kd_tree','brute']

grid = dict(n_neighbors = n_neighbors,
            weights = weights,
            algorithm = algorithm)

grid_search = GridSearchCV(estimator = knn, 
                           param_grid = grid, 
                           cv= 10,  
                           verbose=1, 
                           n_jobs=-1,
                           scoring = "accuracy")

searchResults = grid_search.fit(X_train, y_train.ravel())

# Extract the best model and evaluate it
bestModel = searchResults.best_estimator_

print("Best Parameters (GridSearch):", bestModel)
print("-----------------------------------------------------------")

# Object best hyperparameters
lr = bestModel
  
# Train the model with the best hyperparameters
lr.fit(X_train, y_train)
################################################################################################
y_pred_true_knn = lr.predict(x_test_true)
print("y_pred_true_knn",y_pred_true_knn)
# Confusion matrix
print(confusion_matrix(y_test_true, y_pred_true_knn))
# Classification report
print(classification_report(y_test_true, y_pred_true_knn, digits=3))
mse = mean_squared_error(y_test_true, y_pred_true_knn)
rmse = np.sqrt(mse)
print("RMSE为：", rmse)


Fitting 10 folds for each of 32 candidates, totalling 320 fits
Best Parameters (GridSearch): KNeighborsClassifier(n_neighbors=10, weights='distance')
-----------------------------------------------------------
y_pred_true_knn [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1.]
[[ 2 25]
 [ 2 50]]
              precision    recall  f1-score   support

         0.0      0.500     0.074     0.129        27
         1.0      0.667     0.962     0.787        52

    accuracy                          0.658        79
   macro avg      0.583     0.518     0.458        79
weighted avg      0.610     0.658     0.562        79

RMSE为： 0.5846128222154678


### SVC

In [12]:
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
x_test_true = scaler.transform(x_test_true)

svm = SVC()

# Parameters
gamma =  [0.1, 1.0, 10, 100]
C = [0.1, 1.0, 10, 100]
kernel = ['rbf','linear']

grid = dict(gamma = gamma,
            C = C,
            kernel = kernel)

grid_search = GridSearchCV(estimator = svm, 
                           param_grid = grid, 
                           cv= 10,  
                           verbose=1, 
                           n_jobs=-1,
                           scoring = "accuracy")

searchResults = grid_search.fit(X_train, y_train.ravel())

# Extract the best model and evaluate it
bestModel = searchResults.best_estimator_

print("Best Parameters (GridSearch):", bestModel)
print("-----------------------------------------------------------")

# Object best hyperparameters
svm = bestModel
  
# Train the model with the best hyperparameters
svm.fit(X_train, y_train)
################################################################################################################
pred_svc = svm.predict(x_test_true)

# Confusion matrix
print(confusion_matrix(y_test_true, pred_svc))
# Classification report
print(classification_report(y_test_true, pred_svc,digits=3))
mse = mean_squared_error(y_test_true,pred_svc)
rmse = np.sqrt(mse)
print("RMSE为：", rmse)

Fitting 10 folds for each of 32 candidates, totalling 320 fits
Best Parameters (GridSearch): SVC(C=0.1, gamma=0.1)
-----------------------------------------------------------
[[ 0 27]
 [ 0 52]]
              precision    recall  f1-score   support

         0.0      0.000     0.000     0.000        27
         1.0      0.658     1.000     0.794        52

    accuracy                          0.658        79
   macro avg      0.329     0.500     0.397        79
weighted avg      0.433     0.658     0.523        79

RMSE为： 0.5846128222154678


d:\miniconda\envs\py3.10\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\miniconda\envs\py3.10\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\miniconda\envs\py3.10\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Naive Bayes

In [ ]:
scaler = StandardScaler()   #将数据的均值调整为0，标准差调整为1.
X_train = scaler.fit_transform(x_train)  
x_test_true = scaler.transform(x_test_true)

nb = GaussianNB()

# Parameters
grid = {
    'var_smoothing': np.logspace(0,-9, num=100)
}

# 寻找最优的超参数
grid_search = GridSearchCV(estimator = nb,      #优化模型 朴素贝叶斯
                           param_grid = grid,   #优化参数
                           cv= 10,              #10折线
                           verbose=1,           #日志的详细程度
                           n_jobs=-1,           #使用可用的所有cpu
                           scoring = "accuracy")

searchResults = grid_search.fit(X_train, y_train.ravel())  #.ravel()将多维数组展为一维数组

# Extract the best model and evaluate it
bestModel = searchResults.best_estimator_

print("Best Parameters (GridSearch):", bestModel)
print("-----------------------------------------------------------")

# Object best hyperparameters
nb = bestModel

# Train the model with the best hyperparameters
nb.fit(X_train, y_train)
#######################################################################################################
pred_nb = nb.predict(x_test_true)  #预测标签

# Confusion matrix
print(confusion_matrix(y_test_true, pred_nb))
# Classification report
print(classification_report(y_test_true, pred_nb, digits=3))
mse = mean_squared_error(y_test_true,pred_nb)
rmse = np.sqrt(mse)
print("RMSE为：", rmse)

### Decision Tree

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
x_test_true = scaler.transform(x_test_true)

dt = DecisionTreeClassifier()

# Parameters
max_depth = [2, 3, 5, 10, 20]
min_samples_leaf =  [5, 10, 20, 50, 100]
criterion = ["gini", "entropy"]

grid = dict(max_depth = max_depth,
            min_samples_leaf = min_samples_leaf,
            criterion = criterion)

grid_search = GridSearchCV(estimator = dt, 
                           param_grid = grid, 
                           cv= 10,  
                           verbose=1, 
                           n_jobs=-1,
                           scoring = "accuracy")

searchResults = grid_search.fit(X_train, y_train.ravel())

# Extract the best model and evaluate it
bestModel = searchResults.best_estimator_

print("Best Parameters (GridSearch):", bestModel)
print("-----------------------------------------------------------")

# Object best hyperparameters
dtc = bestModel
  
# Train the model with the best hyperparameters
dtc.fit(X_train, y_train)
###############################################################################
pred_dt = dtc.predict(x_test_true)

# Confusion matrix
print(confusion_matrix(y_test_true, pred_dt))
# Classification report
print(classification_report(y_test_true, pred_dt, digits=3))
mse = mean_squared_error(y_test_true,pred_dt)
rmse = np.sqrt(mse)
print("RMSE为：", rmse)

### Random Forest

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
x_test_true = scaler.transform(x_test_true)

rf = RandomForestClassifier()

# Parameters
bootstrap = [True, False]
max_depth = [10, 20, 50, 100, None]
max_features = ['sqrt', 'log2', None]
min_samples_leaf = [1, 2, 4]
min_samples_split = [2, 5, 10]
n_estimators = [5, 20, 50, 100]

grid = dict(bootstrap = bootstrap, 
            max_depth = max_depth,
            max_features = max_features,
            min_samples_leaf = min_samples_leaf,
            min_samples_split = min_samples_split,
            n_estimators = n_estimators)

grid_search = GridSearchCV(estimator = rf, 
                           param_grid = grid, 
                           cv= 10,  
                           verbose=1, 
                           n_jobs=-1,
                           scoring = "accuracy")

searchResults = grid_search.fit(X_train, y_train.ravel())

# Extract the best model and evaluate it
bestModel = searchResults.best_estimator_

print("Best Parameters (GridSearch):", bestModel)
print("-----------------------------------------------------------")

# Object best hyperparameters
rf = bestModel
  
# Train the model with the best hyperparameters
rf.fit(X_train, y_train)
##########################################################################################################################
pred_RF = rf.predict(x_test_true)

# Confusion matrix
print(confusion_matrix(y_test_true, pred_RF))
# Classification report
print(classification_report(y_test_true, pred_RF, digits=3))
mse = mean_squared_error(y_test_true,pred_RF)
rmse = np.sqrt(mse)
print("RMSE为：", rmse)

### MLP

In [15]:
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
x_test_true = scaler.transform(x_test_true)

mlp = MLPClassifier(max_iter=150)

# Parameters
hidden_layer_sizes = [(10,),(20,),(50,),(100,)]
activation = ['tanh', 'relu']
solver = ['sgd', 'adam']
alpha = [0.0001, 0.05]
learning_rate =  ['constant','adaptive']


grid = dict(hidden_layer_sizes = hidden_layer_sizes,
            activation = activation, 
            solver = solver,
            alpha = alpha,
            learning_rate = learning_rate)

grid_search = GridSearchCV(estimator = mlp, 
                           param_grid = grid, 
                           cv= 10,  
                           verbose=1, 
                           n_jobs=-1,
                           scoring = "accuracy")

searchResults = grid_search.fit(X_train, y_train.ravel())

# Extract the best model and evaluate it
bestModel = searchResults.best_estimator_

print("Best Parameters (GridSearch):", bestModel)
print("-----------------------------------------------------------")

# Object best hyperparameters
lr = bestModel
  
# Train the model with the best hyperparameters
lr.fit(X_train, y_train)
#########################################################################################################
pred_mlp = lr.predict(x_test_true)

# Confusion matrix
print(confusion_matrix(y_test_true, pred_mlp))
# Classification report
print(classification_report(y_test_true, pred_mlp, digits=3))
mse = mean_squared_error(y_test_true,pred_mlp)
rmse = np.sqrt(mse)
print("RMSE为：", rmse)

Fitting 10 folds for each of 64 candidates, totalling 640 fits
Best Parameters (GridSearch): MLPClassifier(alpha=0.05, hidden_layer_sizes=(50,), learning_rate='adaptive',
              max_iter=150, solver='sgd')
-----------------------------------------------------------
[[ 6 21]
 [ 9 43]]
              precision    recall  f1-score   support

         0.0      0.400     0.222     0.286        27
         1.0      0.672     0.827     0.741        52

    accuracy                          0.620        79
   macro avg      0.536     0.525     0.514        79
weighted avg      0.579     0.620     0.586        79

RMSE为： 0.6162360225133208


d:\miniconda\envs\py3.10\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\miniconda\envs\py3.10\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  warnings.warn(


### GBC

In [17]:
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
x_test_true = scaler.transform(x_test_true)

gbc = GradientBoostingClassifier()

# Parameters
learning_rate = [0.01, 0.05, 0.1, 0.15, 0.2]
criterion = ['friedman_mse', 'squared_error']
max_depth = [3,5,8]
max_features = ['log2','sqrt']

grid = dict(learning_rate = learning_rate,
            criterion = criterion,
            max_depth = max_depth,
            max_features = max_features)

grid_search = GridSearchCV(estimator = gbc, 
                           param_grid = grid, 
                           cv= 10,  
                           verbose=1, 
                           n_jobs=-1,
                           scoring = "accuracy")

searchResults = grid_search.fit(X_train, y_train.ravel())

# Extract the best model and evaluate it
bestModel = searchResults.best_estimator_

print("Best Parameters (GridSearch):", bestModel)
print("-----------------------------------------------------------")

# Object best hyperparameters
gbc = bestModel
  
# Train the model with the best hyperparameters
gbc.fit(X_train, y_train)
############################################################################################
pred_gbc = gbc.predict(x_test_true)

# Confusion matrix
print(confusion_matrix(y_test_true, pred_gbc))
# Classification report
print(classification_report(y_test_true, pred_gbc, digits=3))
mse = mean_squared_error(y_test_true,pred_gbc)
rmse = np.sqrt(mse)
print("RMSE为：", rmse)

Fitting 10 folds for each of 60 candidates, totalling 600 fits
Best Parameters (GridSearch): GradientBoostingClassifier(criterion='squared_error', learning_rate=0.01,
                           max_depth=8, max_features='sqrt')
-----------------------------------------------------------
[[ 8 19]
 [ 6 46]]
              precision    recall  f1-score   support

         0.0      0.571     0.296     0.390        27
         1.0      0.708     0.885     0.786        52

    accuracy                          0.684        79
   macro avg      0.640     0.590     0.588        79
weighted avg      0.661     0.684     0.651        79

RMSE为： 0.562543950463012
